# Tool for Time-Series Extraction

This tool requires a dataset of geotiff images as an input. Minor alterations will be required to code depending on naming conventions and extensions. See comments for details

## Import Packages

Tool has numerous required dependcies. Ensure all packages are installed before running.

In [1]:
# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import xarray as xr
from datetime import datetime, timedelta, date
from affine import Affine

# For satellite data
import rasterio as rs
from rasterio.plot import show
import rioxarray
import osgeo
from osgeo import gdal

# Use of open_rasterio gives "DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray." 
# However, rioxarray does not have the transform information necessary to extract coordinates.
import warnings
warnings.filterwarnings('ignore')

## User Inputs

In [14]:
# Glacier name that appears in file name. This will change depending on your personal file naming conventions.

# Jakobshavn Isbrae Glacier in Greenland
glacier = 'JAK'  
hemisphere = 'NORTH'
# Desired extraction point
input_lat = 69.1119; # JAK
input_lon = -49.485; # JAK

# Pine Island Glacier in Antarctica
# glacier = 'PIG'
# hemisphere = 'SOUTH'
# input_lat = -75.3; # PIG
# input_lon = -99.1; # PIG

# Data naming info ----------> These will also need to be adjusted based on the users filename format.
file_extension = '.coffsN_mag_DuFil_yrF.gc.tiff'  
date_format = 'yymmdd'
date_interval_format = 'yymmdd_yymmdd'

## Extract all filenames

In [3]:
# Extract all filenames and create an array of all path names using glob module. 
# This will take a while if it is a large dataset. However, once loaded and saved it does not 
# need to be run again.

# This is the path to where your data is saved
data_path = "/Volumes/b0133/ee21lh/data/velocity/CPOM/"+ glacier + "/T*/*/*/*.coffsN_mag_DuFil_yrF.gc.tiff"

stack_paths = glob.glob(data_path)

np.save(glacier+'_paths', np.array(stack_paths)) # save paths as a numpy array

# Main Code

In [15]:
stack_paths = np.load(glacier+'_paths.npy') # load paths if not already

## Convert extraction cooridinates to WG84 

polar_convert package contains Python functions for converting polar stereographic coordinates. 

Adapted from: https://github.com/nsidc/polarstereo-lonlat-convert-py.

Original software was developed by the NASA National Snow and Ice Data Center Distributed Active Archive Center. Author: Chris Torrence, September 2019

polar_lonlat_to_xy() function was adapted for the northern hemisphere with a central meridian of -45 degrees to create a new function polar_lonlat_north_to_xy().

In [16]:
from polar_convert.constants import SOUTH
from polar_convert.constants import NORTH
from polar_convert import polar_lonlat_to_xy
from polar_convert import polar_lonlat_north_to_xy

true_scale_lat = 71  # true-scale latitude in degrees
re = 6378.137  # earth radius in km
e = 0.08181919 # earth eccentricity

# Convert input point to WGS 84 / Polar Stereographic
if hemisphere == 'NORTH':
    input_point = polar_lonlat_north_to_xy(input_lon, input_lat, true_scale_lat, re, e, hemisphere)
else:
    input_point = polar_lonlat_to_xy(input_lon, input_lat, true_scale_lat, re, e, hemisphere)

input_x,input_y = [input_point[0]*1000,input_point[1]*1000] # convert from km to m

## Extraction Function

For a large dataset this will take quite a while to run. But the print output will ensure it is running correctly.

In [18]:
first_date_index = -(len(file_extension)+len(date_interval_format))
second_date_index = -(len(file_extension)+len(date_format))

# array which data will be put in of form [date1,date2,speed]
time_series = np.empty([len(stack_paths),3]) 

for i in np.arange(0,len(stack_paths)):
    
    # Extracting dates from file names and save to time_series
    date1 = stack_paths[i][first_date_index:first_date_index+len(date_format)]
    time_series[i,0] = date1
    date2 = stack_paths[i][second_date_index:second_date_index+len(date_format)] 
    time_series[i,1] = date2
    
    # Import Geotiff as a DataArray (da) object
    da = xr.open_rasterio(stack_paths[i])
    
    # Extract coordinates
    transform = Affine(*da.attrs["transform"])
    nx, ny = da.sizes["x"], da.sizes["y"]
    x, y = transform * np.meshgrid(np.arange(nx) + 0.5, np.arange(ny) + 0.5)
    
    # Define pandas dataframe
    d = {'x': x.flatten(), 'y': y.flatten(), 'speed (m/yr)': np.array(da).flatten()}
    df = pd.DataFrame(data=d)
    
    # Find point in dataset closest to the input latitude and longitude
    closest_x = df['x'].sub(input_x).abs().idxmin()
    closest_y = df['y'].sub(input_y).abs().idxmin()
    extraction_x,extraction_y = df['x'][closest_x], df['y'][closest_y]
    
    # Extract the speed data at the extraction point and save to time_series
    extract_data = df[(df['x']==extraction_x) & (df['y']==extraction_y)]
    time_series[i,2]=extract_data['speed (m/yr)'].values[0]
    
    
    print('Processed ',i+1,' of ',len(stack_paths), 'images')

time_series_df = pd.DataFrame(time_series,columns = ['Date1','Date2','speed (m/yr)'])
time_series_df.to_csv(glacier+'_speed_time_series.csv',index=False)
                

Processed  1  of  681 images
Processed  2  of  681 images
Processed  3  of  681 images
Processed  4  of  681 images
Processed  5  of  681 images
Processed  6  of  681 images
Processed  7  of  681 images
Processed  8  of  681 images
Processed  9  of  681 images
Processed  10  of  681 images
Processed  11  of  681 images
Processed  12  of  681 images
Processed  13  of  681 images
Processed  14  of  681 images
Processed  15  of  681 images
Processed  16  of  681 images
Processed  17  of  681 images
Processed  18  of  681 images
Processed  19  of  681 images
Processed  20  of  681 images
Processed  21  of  681 images
Processed  22  of  681 images
Processed  23  of  681 images
Processed  24  of  681 images
Processed  25  of  681 images
Processed  26  of  681 images
Processed  27  of  681 images
Processed  28  of  681 images
Processed  29  of  681 images
Processed  30  of  681 images
Processed  31  of  681 images
Processed  32  of  681 images
Processed  33  of  681 images
Processed  34  of  

Processed  269  of  681 images
Processed  270  of  681 images
Processed  271  of  681 images
Processed  272  of  681 images
Processed  273  of  681 images
Processed  274  of  681 images
Processed  275  of  681 images
Processed  276  of  681 images
Processed  277  of  681 images
Processed  278  of  681 images
Processed  279  of  681 images
Processed  280  of  681 images
Processed  281  of  681 images
Processed  282  of  681 images
Processed  283  of  681 images
Processed  284  of  681 images
Processed  285  of  681 images
Processed  286  of  681 images
Processed  287  of  681 images
Processed  288  of  681 images
Processed  289  of  681 images
Processed  290  of  681 images
Processed  291  of  681 images
Processed  292  of  681 images
Processed  293  of  681 images
Processed  294  of  681 images
Processed  295  of  681 images
Processed  296  of  681 images
Processed  297  of  681 images
Processed  298  of  681 images
Processed  299  of  681 images
Processed  300  of  681 images
Processe

Processed  534  of  681 images
Processed  535  of  681 images
Processed  536  of  681 images
Processed  537  of  681 images
Processed  538  of  681 images
Processed  539  of  681 images
Processed  540  of  681 images
Processed  541  of  681 images
Processed  542  of  681 images
Processed  543  of  681 images
Processed  544  of  681 images
Processed  545  of  681 images
Processed  546  of  681 images
Processed  547  of  681 images
Processed  548  of  681 images
Processed  549  of  681 images
Processed  550  of  681 images
Processed  551  of  681 images
Processed  552  of  681 images
Processed  553  of  681 images
Processed  554  of  681 images
Processed  555  of  681 images
Processed  556  of  681 images
Processed  557  of  681 images
Processed  558  of  681 images
Processed  559  of  681 images
Processed  560  of  681 images
Processed  561  of  681 images
Processed  562  of  681 images
Processed  563  of  681 images
Processed  564  of  681 images
Processed  565  of  681 images
Processe